# Similarités entre deux épsiodes

In [46]:
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
from sklearn.feature_extraction import stop_words
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
import re
import math

In [29]:
voc = {'hello', 'bonjour', 'hallo', 'coucou', 'salut'}
v1 = 'hello bonjour salut '
v2 = 'hello salut coucou '
v3 = 'hallo hallo hallo '

texte = v1 + v2 + v3
liste = [v1, v2, v3]

In [30]:
corpus = [texte]
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(corpus)
voc = vectorizer.get_feature_names()
voc

['bonjour', 'coucou', 'hallo', 'hello', 'salut']

In [31]:
v1.split(" ")

['hello', 'bonjour', 'salut', '']

In [32]:
m = []
for t in liste :
    mots = t.split(' ')
    v = []
    for mot in voc :
        if mot in t :
            v.append(1)
        else:
            v.append(0)
    
    m.append(v)
    
m

[[1, 0, 0, 1, 1], [0, 1, 0, 1, 1], [0, 0, 1, 0, 0]]

In [34]:
def distance_euclidienne(v1, v2) :
    d = 0
    for i in range(0, len(v1)) :
        d += (v1[i] - v2[i])**2
    
    return math.sqrt(d)

In [36]:
distance_euclidienne(m[0], m[2])

2.0

In [43]:
d_min = 100000
i_1, i_2 = (0, 0)
for i in range(0, len(m)) :
    v1 = m[i]
    for j in range(0, len(m)) :
        v2 = m[j]
        if( v1 != v2):
            d = distance_euclidienne(v1, v2)
            if d_min > d :
                d_min = d
                i_1 = i
                i_2 = j
                
print(liste[i_1], liste[i_2])

hello bonjour salut  hello salut coucou 


In [89]:
def verifier_ligne(ligne):
    """
    return True si la ligne est un sous-titre, False sinon
    """
    timestamp_regex = r'[0-9]{2}:[0-9]{2}:[0-9]{2}' 
    subnumber_regex =r'^[0-9]+$'
    liste_regex = [timestamp_regex, subnumber_regex]

    for regex in liste_regex:
        if re.match(regex, ligne):
            return False
    return True

def preprocessing_fichier(nom_fichier) :
    """
    string -> string
    à partir du nom d'un fichier de sous-titres, renvoie le texte des sous-titres
    """
    fichier = open(nom_fichier, 'r', encoding = "ISO-8859-1")
    lignes = fichier.readlines()
    fichier.close()
    texte = ""
    for ligne in lignes :
        if verifier_ligne(ligne) :

            m = re.sub(r'[^\w'+"-"']', ' ', ligne)
            
            texte += m
    
    return texte


def scan_folder(parent_folder, corp):
    """
    retourne corpus des textes contenus dans parent_folder sous forme de liste de string
    """
    # iterate over all the files in directory 'parent_folder'
    for file_name in os.listdir(parent_folder):
        if file_name.endswith(".txt"):
            path = parent_folder+"/"+file_name
            
            texte = preprocessing_fichier(path)
            
            corp.append(texte)
        
        else:
            current_path = "".join((parent_folder, "/", file_name))
            if os.path.isdir(current_path):
                # if we're checking a sub-directory, recall this method
                scan_folder(current_path, corp)
    
    return corp


#pour eviter les variables globales
def get_corpus(parent_folder):
    c = []
    res = scan_folder(parent_folder, c)
    return res


def get_hist(df, x_axis, y_axis, titre, colour, font_size=None, horizontal=False):
    """
    
    """
    if horizontal:
        hist = df.plot.barh(x=x_axis, y=y_axis, color=colour, title =titre, fontsize = font_size, edgecolor = "none").get_figure()
    else:
        hist = df.plot.bar(x=x_axis, y=y_axis, color=colour, title =titre, fontsize = font_size, edgecolor = "none").get_figure()
    path_fig = "img/"+titre+'.png'
    hist.savefig(path_fig,  bbox_inches="tight")

In [90]:
corpus = get_corpus("data/175___Avatar__The_Last_Airbender/01") # liste des épisodes
#definition de l'ensemble de stopwords
nltk_sw = set(stopwords.words('english'))
sklearn_sw = set(stop_words.ENGLISH_STOP_WORDS)
stopwords_set = nltk_sw | sklearn_sw
l_nb = [str(i) for i in range(1000000)]
l_mots = ["don", "yeah", "hey", "okay", "oh", "uh", "yes", "ok"]
for mot in l_mots :
    stopwords_set.add(mot)
for nb in l_nb:
    stopwords_set.add(nb)

In [91]:
vectorizer = CountVectorizer(stop_words = stopwords_set)
X = vectorizer.fit_transform(corpus)
voca = vectorizer.get_feature_names() # liste du vocabulaire (en enlevant les stopwords) par ordre alphabétique
len(voca)

3332

In [56]:
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize

In [92]:
ps = PorterStemmer()
voca_stem = set()
for word in voca:
    voca_stem.add(ps.stem(word))
voca_stem = sorted(list(voca_stem))


In [93]:
vectorizer = TfidfVectorizer(stop_words = stopwords_set)
X = vectorizer.fit_transform(corpus)
feature_names = vectorizer.get_feature_names()
dico = vectorizer.get_feature_names()
dense = X.todense()
denselist = dense.tolist()
df_tfidf = pd.DataFrame(denselist, columns=feature_names)
df_tfidf.loc[:,'gonna':].head()

,gonna,good,goodbye,goodness,goodnight,gooey,goofy,got,gotta,gotten,...,yu,yue,yugoda,yup,zhang,zhangs,zhao,zigzags,zone,zuko
0,0.045290,0.064909,0.019577,0.0,0.0,0.0,0.0,0.059264,0.015010,0.00000,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.054517
1,0.058033,0.099806,0.000000,0.0,0.0,0.0,0.0,0.036450,0.030772,0.03171,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000
2,0.024527,0.011717,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.00000,...,0.0,0.0,0.0,0.027785,0.0,0.0,0.0,0.000000,0.0,0.029524
3,0.071558,0.051278,0.010310,0.0,0.0,0.0,0.0,0.037455,0.015810,0.00000,...,0.0,0.0,0.0,0.013511,0.0,0.0,0.0,0.018499,0.0,0.000000
4,0.011463,0.043808,0.000000,0.0,0.0,0.0,0.0,0.071997,0.000000,0.00000,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000


In [94]:
vect = df_tfidf.iloc[[0]]
vect

,16th,41st,aaahh,aaahhh,aag,aahh,aang,aangy,aargh,abandon,...,yu,yue,yugoda,yup,zhang,zhangs,zhao,zigzags,zone,zuko
0,0.0,0.0,0.0,0.0,0.0,0.0,0.134384,0.07025,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.054517


In [96]:
matrix_tfidf = df_tfidf.values

In [117]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=4, random_state=0).fit(matrix_tfidf)
kmeans.labels_

array([3, 3, 0, 3, 0, 3, 3, 2, 3, 0, 3, 3, 0, 3, 2, 2, 0, 0, 1, 0, 3],
      dtype=int32)

In [99]:
def similarity_matrix(m) :
    simil = []
    for i in range(0, len(m)) :
        ep1 = m[i]
        v = []
        for j in range(0, len(m)) :
            ep2 = m[j]
            v.append(-distance_euclidienne(ep1, ep2))
        
        simil.append(v)
    
    return np.array(simil)

In [103]:
sm = similarity_matrix(matrix_tfidf)
len(sm[0])

21

In [107]:
np.argmin(sm, axis=1)

array([18, 18, 18, 11, 11, 18, 18, 18, 11, 18, 18, 18, 18, 18, 11, 18, 18,
       18, 11, 18, 18])